In [1]:
import random
import gym
import gym.spaces
from collections import namedtuple
import numpy as np 
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn 
import torch.optim as optim

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9

In [3]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)
    
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [4]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
    
    def forward(self, x):
        return self.net(x)

In [5]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [6]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [7]:
def filter_batch(batch, percentile):
    filter_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(filter_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)
    
    train_obs = []
    train_act = []
    elite_batch = []
    for exmaple, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, exmaple.steps))
            train_act.extend(map(lambda step: step.action, exmaple.steps))
            elite_batch.append(exmaple)
            
    return elite_batch, train_obs, train_act, reward_bound

In [8]:
random.seed(12345)
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [9]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer = SummaryWriter(comment="-fozenlake-tweaked")

In [10]:
full_batch = []
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]
    
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, rw_mean=%.3f, rw_bound=%.3f, batch=%d"
          % (iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch)))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_mean", reward_mean, iter_no)
    writer.add_scalar("reward_bound", reward_bound, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break

writer.close()

0: loss=1.391, rw_mean=0.030, rw_bound=0.000, batch=3
1: loss=1.388, rw_mean=0.020, rw_bound=0.000, batch=5
2: loss=1.380, rw_mean=0.030, rw_bound=0.000, batch=8
3: loss=1.383, rw_mean=0.030, rw_bound=0.000, batch=11
4: loss=1.381, rw_mean=0.020, rw_bound=0.000, batch=13
5: loss=1.378, rw_mean=0.010, rw_bound=0.000, batch=14
6: loss=1.379, rw_mean=0.020, rw_bound=0.000, batch=16
7: loss=1.377, rw_mean=0.030, rw_bound=0.000, batch=19
8: loss=1.377, rw_mean=0.030, rw_bound=0.000, batch=22
9: loss=1.376, rw_mean=0.010, rw_bound=0.000, batch=23
10: loss=1.377, rw_mean=0.020, rw_bound=0.000, batch=25
11: loss=1.376, rw_mean=0.000, rw_bound=0.000, batch=25
12: loss=1.375, rw_mean=0.010, rw_bound=0.000, batch=26
13: loss=1.373, rw_mean=0.020, rw_bound=0.000, batch=28
14: loss=1.371, rw_mean=0.010, rw_bound=0.000, batch=29
15: loss=1.372, rw_mean=0.010, rw_bound=0.000, batch=30
16: loss=1.371, rw_mean=0.010, rw_bound=0.000, batch=31
17: loss=1.370, rw_mean=0.030, rw_bound=0.000, batch=34
18: l

KeyboardInterrupt: 